In [7]:
require 'open-uri'
require 'json'
require 'daru'
require 'distribution'
require 'sqlite3'
require 'rbplotly'
require "csv"
require './assignment'

include Assignment


Install the spreadsheet gem version ~>1.1.1 for using spreadsheet functions.

Install the mechanize gem version ~>2.7.5 for using mechanize functions.


"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

Object

In [8]:
dir = "#{ENV['HOME']}/cs6140/final_project"
db = SQLite3::Database.new "#{dir}/credit_risk_data.db"
db.results_as_hash = true

true

In [9]:
def fetch_train_data db, features, limit = true
  table = "application_train"
  query = "select " + features.join(",") + ", Target label from " + table
  query += " order by RANDOM() limit 10000" if limit
  data = []
  db.execute(query).each do |r|
    row = Hash.new
    row["features"] = Hash.new
    features.each do |f|
      row["features"][f] = r[f]
    end
    row["label"] = r["label"]
    data << row
  end
  return data
end


def fetch_test_data db, features
  features << "SK_ID_CURR"
  table = "application_test"
  query = "select " + features.join(",") + " from " + table
  data = []
  db.execute(query).each do |r|
    row = Hash.new
    row["features"] = Hash.new
    features.each do |f|
      row["features"][f] = r[f]
    end
    row["label"] = r["label"]
    data << row
  end
  return data
end

:fetch_test_data

## Step 2.  Actual Code

In [10]:
# return the minimum support in order to generate a rule
def min_supp data, label, minsups
  x_class = parse_class data
  return minsups[label] * x_class[label].to_f / x_class.values.sum
end

:min_supp

In [11]:
# Generate the n-label linguistic fuzzy sets based on all possible features
def feature_fuzzy_sets data, n
  fs = []
  h = data[0]["features"].keys
  h.each do |f|
    scores = data.collect {|x| x["features"][f]}.select{ |x| x && x!= "" }.
      sort.uniq
    # 5-label fuzzy set
    (0..(n-1)).each do |i|
      max = scores.max
      min = scores.min
      # since min is 0 and max is 1, there will be only 4 gaps
      base = (max - min).to_f / (n-1)
      r = Hash.new
      r["feature"] = f
      r["set"] = i
      r["mid"] = min + base * i
      r["base"] = base * 2
      fs << r
    end
  end
  return fs
end

:feature_fuzzy_sets

In [12]:
def support data, antes, cl, mu_table
  u = 0.0
  data.each do |x|
    next if x["label"] != cl
    mus = []
    antes.each do |ante|
      v = x["features"][ante["feature"]]
      next if v.nil? or v == ""
      header = ante["feature"].to_s + "_" + ante["set"].to_s
      mu = mu_table[header][v]
      mus << mu
      # skip some iterations if we found one mu as 0.
      break if mu == 0
    end
    u += t_norm mus if mus.length > 0
  end
  return u/data.length
end

def confidence data, antes, cl, mu_table
  u = 0.0
  d = 0.0
  data.each do |x|
    mus = []
    antes.each do |ante|
      v = x["features"][ante["feature"]]
      next if v.nil? or v == ""
      header = ante["feature"].to_s + "_" + ante["set"].to_s
      mu = mu_table[header][v]
      mus << mu
      # skip some iterations if we found one mu as 0.
      break if mu == 0
    end
    u += t_norm mus if x["label"] == cl && mus.length > 0
    d += t_norm mus if mus.length > 0
  end
  return u/d
end

def mu x, ante, w, negative_enabled = false
  f = ante["feature"]
  mid = ante["mid"]
  base = ante["base"]
  v = x["features"][f]
  puts "gaga" if v == "" or v.nil?
  mem = 1.0 - ((v - (mid + w * ante["base"])).to_f * 2 / base).abs
  return mem if negative_enabled
  return [0.0, mem].max
end

# Calulate the t-normalization of a vecotr x, use either min or product t-norm
def t_norm x
  return x.min
end

def t_norm x
  return x.reduce(:*)
end

:t_norm

In [13]:
def supp_prune rules, min_supp
  return rules.select { |r| r["support"] > min_supp}
end

def confi_prune rules
  return rules.select { |r| r["confidence"] > 0.0 and r["confidence"] < 1.0 }
end

def generate_leaves data, last_leaves, first_fuzzy, mu_table, cl, log
  leaves = []
  leaves_names = []
  puts "\t Last Leaves number " + last_leaves.length.to_s if log
  last_leaves.each.with_index do |rule1, i|
    print "Processing Leaf " + i.to_s + "\r"
    f1 = rule1["antecedent"].map { |r| r["feature"] }
    first_fuzzy.each do |rule2|
      f2 = rule2["antecedent"].map { |r| r["feature"] }
      total = (f1 + f2).uniq.length
      
      next if total < f1.length + f2.length
      next if leaves_names.include? (f1 + f2) 
      
      rule = Hash.new
      rule["antecedent"] = (rule1["antecedent"] + rule2["antecedent"])
      rule["support"] = support data, rule["antecedent"], cl, mu_table
      rule["confidence"] = confidence data, rule["antecedent"], cl, mu_table
      rule["class"] = cl
      leaves << rule
      leaves_names << f1 + f2
    end
  end
  puts
  return leaves
end

def generate_mu_table data, ffs, w
  mu_table = Hash.new

  # Initialize the rules
  ffs.each do |fs|
    (0..1).each do |cl|
      header = fs["feature"].to_s + "_" + fs["set"].to_s
      mu_table[header] = Hash.new
      data.each do |x|
        v = x["features"][fs["feature"]]
        mu_table[header][v] = (mu x, fs, w[header]) if !mu_table[header].key?(v) and v != "" and !v.nil?
      end
    end
  end
  return mu_table
end

:generate_mu_table

In [14]:
# Build a search tree based on five-label fuzzy set
def build_search_tree data, min_supps, max_lvl, n, w, log = true
  # Generate the first layer of rules, getting all possible fuzzy sets by the 5-label linguistic rule
  last_leaves = Hash.new { |h, k| h[k] = []}
  first_ffs = Hash.new { |h, k| h[k] = []}
  ffs = feature_fuzzy_sets data, n
  
  puts "Initialize mu table" if log
  
  mu_table = generate_mu_table data, ffs, w
  
  puts "Mu table initialization complete" if log
  puts "Generate first lvl of leaves" if log
  
  ffs.each do |fs|
    (0..1).each do |cl|
      rule = Hash.new
      rule["antecedent"] = [fs]
      rule["support"] = support data, rule["antecedent"], cl, mu_table
      rule["confidence"] = confidence data, rule["antecedent"], cl, mu_table
      rule["class"] = cl
      last_leaves[cl] << rule
      first_ffs[cl] << rule
    end
  end
  
  rules = Hash.new { |h, k| h[k] = []}
 
  (0..1).each do |cl|
    curr_lvl = 0
    print "Genearte search tree for label ", cl, "\n" if log
    while true
      curr_lvl += 1
      print "\t lvl ", curr_lvl, "\n" if log
      first_ffs[cl] = (supp_prune first_ffs[cl], min_supps[cl]) if curr_lvl == 1
      last_leaves[cl] = (supp_prune last_leaves[cl], min_supps[cl])

      rules[cl] = last_leaves[cl]
      break if curr_lvl >= max_lvl || last_leaves[cl].length <= 2
      last_leaves[cl] = (confi_prune last_leaves[cl])
      last_leaves[cl] = generate_leaves data, last_leaves[cl], first_ffs[cl], mu_table, cl, log
    end
  end
  
  return rules
end

:build_search_tree

In [15]:
def mu_iv x, ante, w
  f = ante["feature"]
  mid = ante["mid"]
  base = ante["base"]
  v = x["features"][f]
  puts "gaga" if v == ""  
  mem = 1.0 - ((v - (mid + w * ante["base"])).to_f * 2 / base).abs
  mem_up = 1.0 - ((v - (mid + w * ante["base"])).to_f * 2 / (base*1.5)).abs
  return [0.0, mem].max, [0.0, mem_up].max
end

def generate_mu_iv_table data, ffs, w
  mu_table = Hash.new
  # Initialize the rules
  ffs.each do |fs|
    (0..1).each do |cl|
      header = fs["feature"].to_s + "_" + fs["set"].to_s
      mu_table[header] = Hash.new
      data.each do |x|
        v = x["features"][fs["feature"]]
        if !mu_table[header].key?(v) and v != "" and !v.nil?
          mu_table[header][v] = Hash.new         
          mu_table[header][v]["lo"], mu_table[header][v]["hi"] = (mu_iv x, fs, w[header])
        end
      end
    end
  end
  return mu_table
end

:generate_mu_iv_table

In [16]:
def predict_iv x, rbs
  votes = Hash.new { |h,k| h[k] = 0.0 }
  (0..1).each do |cl|
    rbs.each do |r|
      next if r["class"] != cl
      r["antecedent"].each do |ante|
        v = x["features"][ante["feature"]]
        next if v.nil? or v == ""
        header = ante["feature"].to_s + "_" + ante["set"].to_s
        lo, hi = (mu_iv x, ante, 0.0)
        votes[cl] += r["confidence"] * ( hi + lo ) *  hi / r["support"]
      end
    end
  end
  return votes
end

:predict_iv

In [17]:
# ffs = feature_fuzzy_sets data_fuzzy_train, 5
  
# puts "Initialize mu table"
# mu_iv_table = generate_mu_iv_table data_fuzzy_train, ffs, w
# puts "Mu table initialization complete"
features_all = ["EXT_SOURCE_3", "EXT_SOURCE_1", "EXT_SOURCE_2", "DAYS_BIRTH", "DAYS_LAST_PHONE_CHANGE",  
  "AMT_GOODS_PRICE", "OWN_CAR_AGE",  "REG_CITY_NOT_WORK_CITY", "FLAG_EMP_PHONE", "REGION_RATING_CLIENT_W_CITY",
  "DAYS_ID_PUBLISH", "REGION_RATING_CLIENT", "FLAG_DOCUMENT_3", "REG_CITY_NOT_LIVE_CITY", "REGION_POPULATION_RELATIVE",
  "AMT_CREDIT"]
data_all = fetch_train_data db, features_all, false 
1

1

In [49]:
features_all = ["EXT_SOURCE_3", "EXT_SOURCE_1", "EXT_SOURCE_2", "DAYS_BIRTH", "DAYS_LAST_PHONE_CHANGE",  "AMT_GOODS_PRICE", "OWN_CAR_AGE", 
  "REG_CITY_NOT_WORK_CITY", "FLAG_EMP_PHONE", "REGION_RATING_CLIENT_W_CITY", "DAYS_ID_PUBLISH", "REGION_RATING_CLIENT", "FLAG_DOCUMENT_3",
  "REG_CITY_NOT_LIVE_CITY", "REGION_POPULATION_RELATIVE", "AMT_CREDIT"]
features_comb = (features_all.combination(3).to_a)
n = 20
w = Hash.new { |h,k| h[k] = 0.0 }
rbss = []

(1..n).each do |i|
  #Randomly sampling 10000 instances
  features = features_comb.sample
  data = fetch_train_data db, features
  
  # calculate the minimum support
  minsups = [0.1, 0.12]
  min_supps = Hash.new
  min_supps[0] = min_supp data, 0, minsups
  min_supps[1] = min_supp data, 1, minsups
  
  # max lvl of the tree
  max_lvl = 3
  
  rb = []
  rb = build_search_tree data, min_supps, max_lvl, 5, w, false
  rbs = rb[0] + rb[1]
  rbss << rbs
  puts "Finish building model " + i.to_s
end
1

Processing Leaf 11
Processing Leaf 43
Processing Leaf 10
Processing Leaf 43
Finish building model 1
Processing Leaf 12
Processing Leaf 29
Processing Leaf 11
Processing Leaf 21
Finish building model 2
Processing Leaf 11
Processing Leaf 29
Processing Leaf 13
Processing Leaf 23
Finish building model 3
Processing Leaf 12
Processing Leaf 45
Processing Leaf 12
Processing Leaf 29
Finish building model 4
Processing Leaf 11
Processing Leaf 51
Processing Leaf 9
Processing Leaf 39
Finish building model 5
Processing Leaf 11
Processing Leaf 41
Processing Leaf 9
Processing Leaf 29
Finish building model 6
Processing Leaf 11
Processing Leaf 51
Processing Leaf 9
Processing Leaf 31
Finish building model 7
Processing Leaf 12
Processing Leaf 51
Processing Leaf 10
Processing Leaf 35
Finish building model 8
Processing Leaf 11
Processing Leaf 57
Processing Leaf 9
Processing Leaf 29
Finish building model 9
Processing Leaf 14
Processing Leaf 35
Processing Leaf 12
Processing Leaf 19
Finish building model 10
Pro

1

In [12]:
# now it's the time to do actual predictions
predictions = []
data_test = data_all[0, 10000]

data_test.each.with_index do |row, i|
  total = 0.0
  rbss.each.with_index do |rbs, i|
    p = predict_iv row, rbs
    total += (p[1]/(p[0] + 0.01)) if p.length > 1
  end
  print "Processing data " + i.to_s + "\r"
  predictions << { "something" => total }
end

scores = predictions.collect{ |k| k.values[0] }.uniq.sort.reverse
scores_limited = scores.select.with_index {|x, i| i % 200 == 0}
1

NameError: undefined local variable or method `rbss' for main:Object

In [13]:
fprs, tprs = roc_curve data_test, predictions, scores_limited
plot(fprs, tprs)

NoMethodError: undefined method `each' for nil:NilClass

In [14]:
auc fprs, tprs 

NoMethodError: undefined method `each' for nil:NilClass

In [53]:
features_all.combination(2).to_a.length

120

In [54]:
data_real = fetch_test_data db, features_all
1

1

In [55]:
# now it's the time to do actual predictions
predictions = []

data_real.each.with_index do |row, i|
  total = 0.0
  rbss.each.with_index do |rbs, i|
    p = predict_iv row, rbs
    total += (1e-3 * p[1] / (p[0] + 0.01)) if p.length > 1
  end
  print "Processing data " + i.to_s + "\r"
  predictions << { row["features"]["SK_ID_CURR"] => total }
end
1

1

In [ ]:
data_real[0]

In [56]:
CSV.open("submission.csv", "wb") do |csv|
  csv << ["SK_ID_CURR", "TARGET"]
  predictions.each do |p|
    p.each do |k, v|
      csv << [k, v]
    end
  end
end
1

1

In [31]:
data_real = fetch_test_data db, features_all
1

1

In [ ]:
# single IVFS case
features = ["EXT_SOURCE_3", "EXT_SOURCE_1", "EXT_SOURCE_2", "DAYS_BIRTH", "DAYS_LAST_PHONE_CHANGE",  "AMT_GOODS_PRICE", "OWN_CAR_AGE", 
  "REG_CITY_NOT_WORK_CITY", "FLAG_EMP_PHONE", "REGION_RATING_CLIENT_W_CITY", "DAYS_ID_PUBLISH", "REGION_RATING_CLIENT", "FLAG_DOCUMENT_3",
  "REG_CITY_NOT_LIVE_CITY", "REGION_POPULATION_RELATIVE", "AMT_CREDIT"]
#features = ["EXT_SOURCE_3", "EXT_SOURCE_1", "EXT_SOURCE_2", "DAYS_BIRTH"]
data = fetch_train_data db, features
data = data[0, 10000]
w = Hash.new { |h,k| h[k] = 0.0 }  

# calculate the minimum support
minsups = [0.05, 0.06]
min_supps = Hash.new
min_supps[0] = min_supp data, 0, minsups
min_supps[1] = min_supp data, 1, minsups
  
# max lvl of the tree
max_lvl = 3

rb = []
rb = build_search_tree data, min_supps, max_lvl, 3, w, true  
rbs = rb[0] + rb[1]
1

Initialize mu table
Mu table initialization complete
Generate first lvl of leaves


In [ ]:
rbs.length

In [ ]:
puts rbs.length

In [ ]:
puts rb[0].length, rb[1].length

In [ ]:
def predict_iv x, rbs
  votes = Hash.new { |h,k| h[k] = 0.0 }
  (0..1).each do |cl|
    rbs.each do |r|
      next if r["class"] != cl
      r["antecedent"].each do |ante|
        v = x["features"][ante["feature"]]
        next if v.nil? or v == ""
        header = ante["feature"].to_s + "_" + ante["set"].to_s
        lo, hi = (mu_iv x, ante, 0.0)
        votes[cl] += r["confidence"] * ( hi + lo ) *  hi
      end
    end
  end
  return votes
end

predictions = []

data_test = data_all.shuffle[0, 1000]

data_test.each.with_index do |x, i|
  total = 0.0
  s = predict_iv x, rbs
  max = [s[0], s[1]].max
  predictions << { s.key(max) => s[1] / (s[0] + 0.01) }
  print "Processing data " + i.to_s + "\r"
end

scores = predictions.collect{ |k| k.values[0] }.uniq.sort.reverse
scores_limited = scores.select.with_index {|x, i| i % 10 == 0}
puts scores[0, 10]
1

In [35]:
fprs, tprs = roc_curve data_test, predictions, scores_limited
puts fprs[0, 10]
puts tprs[0, 10]
plot(fprs, tprs)

[0.0, 0.0, 0.009825327510917031, 0.017467248908296942, 0.026200873362445413, 0.034934497816593885, 0.043668122270742356, 0.05131004366812227, 0.05895196506550218, 0.06768558951965066]
[0.0, 0.0, 0.011904761904761904, 0.047619047619047616, 0.07142857142857142, 0.09523809523809523, 0.11904761904761904, 0.15476190476190477, 0.19047619047619047, 0.21428571428571427]


#<Nyaplot::Plot:0x00000000870c7a38 @properties={:diagrams=>[#<Nyaplot::Diagram:0x0000000087026fc0 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"57c9deb9-2f2b-40e0-a24b-ad7bb53609f3"}, @xrange=[0.0, 1.0], @yrange=[0.0, 1.0]>], :options=>{:x_label=>"X", :y_label=>"Y", :zoom=>true, :width=>700, :xrange=>[0.0, 1.0], :yrange=>[0.0, 1.0]}}>

In [37]:
auc fprs, tprs 

0.6406217508837594

In [ ]:
# now it's the time to do actual predictions
predictions = []

data_real.each.with_index do |row, i|
  total = 0.0
  s = predict_iv row, rbs
  s[0] = s[0] * 0.015
  print "Processing data " + i.to_s + "\r"
  predictions << { row["features"]["SK_ID_CURR"] => 1e-3 * s[1] / (s[0] + 0.01) }
end
1

In [ ]:
CSV.open("submission.csv", "wb") do |csv|
  csv << ["SK_ID_CURR", "TARGET"]
  predictions.each do |p|
    p.each do |k, v|
      csv << [k, v]
    end
  end
end
1

In [58]:
def set_to_ling set
  if set == 0
    return "Very Low"
  elsif set == 1
    return "Low"
  elsif set == 2
    return "Medium"
  elsif set == 3
    return "High"
  elsif set == 4
    return "Very High"
  end
end

def set_to_ling set
  if set == 0
    return "Low"
  elsif set == 1
    return "Medium"
  elsif set == 2
    return "High"
  end
end

# print the rule base
def print_rbs rbs, i, n, best = true
  rs = ""
  rbs.each do |rbs_l|
    rbs_l = best ? rbs_l.sort_by { |r| -r["confidence"] }[0, i] : rbs_l.sort_by { |r| -r["confidence"] }.reverse[0, i]
    rs += "Class " + rbs_l[0]["class"].to_s + "\n"
    
    rbs_l.each.with_index do |r, i|
      s = "\trule " + (i+1).to_s + ": If "
      r["antecedent"].each do |ante|
        s += ante["feature"].to_s + " is " + (set_to_ling ante["set"])
        s += " and " if ante != r["antecedent"].last
      end
      s += " Then Class is " + r["class"].to_s + " with Confidence " + r["confidence"].round(n).to_s +
        " and Support " + r["support"].round(n).to_s
      rs += s + "\n"
    end
  end
  return rs
end

:print_rbs

In [59]:
puts "Best 10 rules\n"
puts print_rbs [rb[0], rb[1]], 5, 5
puts "Worst 10 rules\n"
puts "split line----------------------------------------------------------\n"
puts print_rbs [rb[0], rb[1]], 5, 5, false

Best 10 rules

Class 0
	rule 1: If EXT_SOURCE_2 is High and EXT_SOURCE_3 is High and REGION_RATING_CLIENT is Medium Then Class is 0 with Confidence 0.96673 and Support 0.09476
	rule 2: If EXT_SOURCE_3 is High and REGION_RATING_CLIENT is Medium and EXT_SOURCE_2 is High Then Class is 0 with Confidence 0.96673 and Support 0.09476
	rule 3: If EXT_SOURCE_3 is High and EXT_SOURCE_2 is High and REGION_RATING_CLIENT is Medium Then Class is 0 with Confidence 0.96673 and Support 0.09476
	rule 4: If REGION_RATING_CLIENT is Medium and EXT_SOURCE_3 is High and EXT_SOURCE_2 is High Then Class is 0 with Confidence 0.96673 and Support 0.09476
	rule 5: If EXT_SOURCE_2 is High and REGION_RATING_CLIENT is Medium and EXT_SOURCE_3 is High Then Class is 0 with Confidence 0.96673 and Support 0.09476
Class 1
	rule 1: If FLAG_EMP_PHONE is High and DAYS_LAST_PHONE_CHANGE is High and EXT_SOURCE_2 is Low Then Class is 1 with Confidence 0.20539 and Support 0.01318
	rule 2: If DAYS_LAST_PHONE_CHANGE is High and EXT

In [32]:
puts rb[0].length

12933


In [33]:
puts rb[1].length

9725


In [4]:
[["hello", "world"],["hello", "world"]].include? ["hello", "waorld"]

false